In [16]:
from pyspark.sql import SparkSession

# Cria uma SparkSession
spark = (SparkSession.builder
         .appName("MinhaAppSpark")
         .master("local[*]")  # "local[*]" usa todos os núcleos disponíveis localmente
         .config("spark.ui.port", "4040") 
         .getOrCreate()
         )
spark.sparkContext.setLogLevel("ERROR")

# Verifique se a sessão foi criada
print(spark.version)

3.5.3


In [17]:
import sys
import os

# Adiciona o diretório atual ao Python path
sys.path.append(os.path.dirname(os.getcwd()))
   
from libs.storage import Storage

storage_log = Storage("files/sipe-u-log-hist-ate-2025")
storage_pefil = Storage("files/perfil")
storage_data = Storage("data")
storage_servidor = Storage("files/siape-servidor")

limit = 10000
random_seed = 42
storage_log

Storage(account_name='', container_name='', directory_name='files/sipe-u-log-hist-ate-2025', directory_path='/home/lima/projetos/treinamento/bootML/', local='local')

In [18]:
colunas = ["co_orgao_log","classificacao", "SIGLA_SISTEMA","SUBSISTEMA","MODULO","OPCAO","ATIVIDADE","TRANSACAO"]

In [26]:
df = spark.read.parquet(storage_data.get_path(f"dataset"))
df.dropna(subset=["SIGLA_SISTEMA"])
df.printSchema()
df.show(3)

root
 |-- isn: long (nullable = true)
 |-- co_orgao_log: long (nullable = true)
 |-- ch_arvore_senha_log: string (nullable = true)
 |-- da_atualizacao_log: string (nullable = true)
 |-- ho_atualizacao_log: long (nullable = true)
 |-- nu_cpf_usuario_log: long (nullable = true)
 |-- co_terminal_log: string (nullable = true)
 |-- co_crh_log: long (nullable = true)
 |-- da_ano_mes_pagamento: integer (nullable = true)
 |-- nu_iden_unica_siape_log: integer (nullable = true)
 |-- matricula_inst_legal: long (nullable = true)
 |-- co_orgao_servidor: integer (nullable = true)
 |-- classificacao: string (nullable = true)
 |-- SIGLA_SISTEMA: string (nullable = true)
 |-- SUBSISTEMA: string (nullable = true)
 |-- MODULO: string (nullable = true)
 |-- OPCAO: string (nullable = true)
 |-- ATIVIDADE: string (nullable = true)
 |-- TRANSACAO: string (nullable = true)

+-------+------------+--------------------+------------------+------------------+------------------+---------------+----------+----------

In [13]:
from pyspark.sql.functions import col, monotonically_increasing_id, when, countDistinct


def criar_mapeamento(df, coluna):
    """
    Cria um dicionário mapeando valores únicos de uma coluna para números sequenciais.

    Args:
        df: O DataFrame Spark.
        coluna: A coluna a ser mapeada.

    Returns:
        Um dicionário com o mapeamento.
    """

    df_temp = df.select(coluna).distinct().withColumn("novo_valor", monotonically_increasing_id() + 1)
    return df_temp.rdd.map(lambda row: (row[coluna], row["novo_valor"])).collectAsMap()

# Exemplo de uso:
def aplicar_mapeamento(df, coluna, mapeamento):
    """
    Substitui os valores de uma coluna por seus valores mapeados.

    Args:
        df: O DataFrame Spark.
        coluna: A coluna a ser modificada.
        mapeamento: O dicionário com o mapeamento.
    """

    expr = col(coluna)
    for valor_origem, valor_destino in mapeamento.items():
        expr = when(col(coluna) == valor_origem, valor_destino).otherwise(expr)
    return df.withColumn(f"map_{coluna}", expr)



In [14]:
#  |-- SIGLA_SISTEMA: string (nullable = true)
#  |-- SUBSISTEMA: string (nullable = true)
#  |-- MODULO: string (nullable = true)
#  |-- OPCAO: string (nullable = true)
#  |-- ATIVIDADE: string (nullable = true)
#  |-- TRANSACAO: string (nullable = true)

col_sis = "SIGLA_SISTEMA"
col_sub = "SUBSISTEMA"
col_mod = "MODULO"
col_opc = "OPCAO"
col_ati = "ATIVIDADE"
col_tra = "TRANSACAO"

# [col_sis]
mapeamento = criar_mapeamento(df, col_sis)
df = aplicar_mapeamento(df, col_sis, mapeamento)

df_sis_dist = df.select(col_sis).distinct()
for row_sis in df_sis_dist.collect():
    id = row_sis[col_sis]
    df_sub = df.where(f'{col_sis} = {id}')
    
    # [col_sis][col_sub]
    mapeamento = criar_mapeamento(df_sub, col_sub)
    df = aplicar_mapeamento(df, col_sub, mapeamento)
    
    df_sub_dist = df_sub.select(col_sub).distinct()
    for row_sub in df_sub_dist.collect():
        id = row_sub[col_sub]
        df_mod = df_sub.where(f'{col_sub} = {id}')
        
        # [col_sis][col_sub][col_mod]
        mapeamento = criar_mapeamento(df_mod, col_mod)
        df = aplicar_mapeamento(df, col_mod, mapeamento)
        
        df_mod_dist = df_mod.select(col_mod).distinct()
        for row_mod in df_mod_dist.collect():
            id = row_mod[col_mod]
            df_opc = df_sub.where(f'{col_mod} = {id}')
            
            # [col_sis][col_sub][col_mod][col_opc]
            mapeamento = criar_mapeamento(df_opc, col_opc)
            df = aplicar_mapeamento(df, col_opc, mapeamento)
            
            df_opc_dist = df_opc.select(col_opc).distinct()
            for row_opc in df_opc_dist.collect():
                id = row_opc[col_opc]
                df_ati = df_sub.where(f'{col_opc} = {id}')
                
                # [col_sis][col_sub][col_mod][col_opc][col_ati]
                mapeamento = criar_mapeamento(df_ati, col_ati)
                df = aplicar_mapeamento(df, col_ati, mapeamento)
                
                df_ati_dist = df_ati.select(col_ati).distinct()
                for row_ati in df_ati_dist.collect():
                    id = row_ati[col_ati]
                    df_tra = df_sub.where(f'{col_opc} = {id}')
                    
                    # [col_sis][col_sub][col_mod][col_opc][col_ati][col_tra]
                    mapeamento = criar_mapeamento(df_tra, col_tra)
                    df = aplicar_mapeamento(df, col_tra, mapeamento)
                    
                    df_tra_dist = df_tra.select(col_ati).distinct()
                    for row_tra in df_tra_dist.collect():
                        id = row_tra[col_tra]
                        df_tra = df_sub.where(f'{col_tra} = {id}')
                        
                        mapeamento = criar_mapeamento(df_tra, col_tra)
                        df = aplicar_mapeamento(df, col_tra, mapeamento)


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `EORG` cannot be resolved. Did you mean one of the following? [`isn`, `OPCAO`, `MODULO`, `TRANSACAO`, `ATIVIDADE`].; line 1 pos 16;
'Filter (SIGLA_SISTEMA#97 = 'EORG)
+- Project [isn#84L, co_orgao_log#85L, ch_arvore_senha_log#86, da_atualizacao_log#87, ho_atualizacao_log#88L, nu_cpf_usuario_log#89L, co_terminal_log#90, co_crh_log#91L, da_ano_mes_pagamento#92, nu_iden_unica_siape_log#93, matricula_inst_legal#94L, co_orgao_servidor#95, classificacao#96, SIGLA_SISTEMA#97, SUBSISTEMA#98, MODULO#99, OPCAO#100, ATIVIDADE#101, TRANSACAO#102, CASE WHEN (SIGLA_SISTEMA#97 = ) THEN cast(5 as string) ELSE CASE WHEN (SIGLA_SISTEMA#97 = SIAPECAD) THEN cast(4 as string) ELSE CASE WHEN (SIGLA_SISTEMA#97 = FOLHA) THEN cast(3 as string) ELSE CASE WHEN (SIGLA_SISTEMA#97 = SIAPE) THEN cast(2 as string) ELSE CASE WHEN (SIGLA_SISTEMA#97 = EORG) THEN cast(1 as string) ELSE SIGLA_SISTEMA#97 END END END END END AS m

In [27]:
df.show()


+--------+------------+--------------------+------------------+------------------+------------------+---------------+----------+--------------------+-----------------------+--------------------+-----------------+-------------+-------------+----------+---------+-----+---------+---------+
|     isn|co_orgao_log| ch_arvore_senha_log|da_atualizacao_log|ho_atualizacao_log|nu_cpf_usuario_log|co_terminal_log|co_crh_log|da_ano_mes_pagamento|nu_iden_unica_siape_log|matricula_inst_legal|co_orgao_servidor|classificacao|SIGLA_SISTEMA|SUBSISTEMA|   MODULO|OPCAO|ATIVIDADE|TRANSACAO|
+--------+------------+--------------------+------------------+------------------+------------------+---------------+----------+--------------------+-----------------------+--------------------+-----------------+-------------+-------------+----------+---------+-----+---------+---------+
| 4250438|          13|SIAPE     SIAPENE...|        2024-02-13|             84837|                18|        NATUSER|   1389761|        

In [28]:
df_test = df[colunas].toPandas()
df_test

,co_orgao_log,classificacao,SIGLA_SISTEMA,SUBSISTEMA,MODULO,OPCAO,ATIVIDADE,TRANSACAO
0,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO
1,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO
2,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO
3,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO
4,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO
...,...,...,...,...,...,...,...,...
9995,2,Coordenador Geral,,,,,,
9996,2,Coordenador Geral,,,,,,
9997,2,Coordenador Geral,,,,,,
9998,2,Coordenador Geral,,,,,,


In [29]:
df_test.value_counts()

co_orgao_log  classificacao      SIGLA_SISTEMA  SUBSISTEMA  MODULO     OPCAO      ATIVIDADE  TRANSACAO           
1             Coordenador Geral  SIAPE          FOLHA       ATUASERV                         FPATPEQMSE              862
                                                                                                                     494
6             Coordenador Geral  SIAPE          SIAPE       GERENCIAL                        GRPROCESSOGRDESBLQRH    414
3             Chefe                                                                                                  404
25            Diretor            SIAPE          CADSIAPE    CADASTRO   ATUCADAST             CDALCONPGDN92116W4      302
                                                                                                                    ... 
1             Coordenador Geral  SIAPE          SIAPECAD    DADOSFUNC  LOTACAO    LOCALEXER  CALCEXERIN                1
                                       

In [30]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler  # Import OrdinalEncoder and StandardScaler
from sklearn.cluster import mean_shift

ordinal_encoder = OrdinalEncoder()

# Create a transformation pipeline
pipeline = Pipeline([
    ('ordinal_encoder', ordinal_encoder),
])

X_transformed = pipeline.fit_transform(df_test)
print(X_transformed.shape)
X_transformed

(10000, 8)


array([[12.,  3.,  3., ...,  0.,  0., 83.],
       [12.,  3.,  3., ...,  0.,  0., 83.],
       [12.,  3.,  3., ...,  0.,  0., 83.],
       ...,
       [ 1.,  2.,  0., ...,  0.,  0.,  0.],
       [ 1.,  2.,  0., ...,  0.,  0.,  0.],
       [ 1.,  2.,  0., ...,  0.,  0.,  0.]])

In [31]:
import numpy as np
from sklearn.cluster import mean_shift
X = X_transformed
cluster_centers, labels = mean_shift(X, bandwidth=4)
print(cluster_centers)
labels

[[ 2.08284884  1.19331395  0.          0.          0.          0.
   0.          0.        ]
 [ 0.11611161  1.91179118  2.93339334  5.          3.          0.
   0.         69.90729073]
 [ 4.02719855  1.69990934  3.         12.         14.          0.
   0.          5.        ]
 [ 5.08524904  1.97509579  3.         10.97509579 14.51245211  0.
   0.         81.16475096]
 [10.24859551  2.74438202  3.         12.         14.          0.
   0.          5.        ]
 [ 6.60940032  1.63209076  2.88006483  5.          3.          0.
   0.         70.31766613]
 [22.73975045  2.67736185  3.          2.          5.          3.
   0.         38.        ]
 [ 4.13984674  0.75670498  3.          2.          7.          0.
   0.         47.3467433 ]
 [10.96289062  2.77734375  3.         12.         14.          0.
   0.         83.        ]
 [ 6.62555066  0.90528634  0.          0.          0.          0.
   0.          0.        ]
 [ 4.61764706  0.77149321  3.          5.          9.00678733  0.
   0

array([8, 8, 8, ..., 0, 0, 0])

In [32]:
df_test["target"] = labels
df_test

,co_orgao_log,classificacao,SIGLA_SISTEMA,SUBSISTEMA,MODULO,OPCAO,ATIVIDADE,TRANSACAO,target
0,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO,8
1,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO,8
2,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO,8
3,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO,8
4,13,Diretor,SIAPE,SIAPENET,FERIASWEB,,,INCLUSAO,8
...,...,...,...,...,...,...,...,...,...
9995,2,Coordenador Geral,,,,,,,0
9996,2,Coordenador Geral,,,,,,,0
9997,2,Coordenador Geral,,,,,,,0
9998,2,Coordenador Geral,,,,,,,0


In [65]:
df_test.value_counts().head(20)

co_orgao_log  classificacao      SIGLA_SISTEMA  SUBSISTEMA  MODULO     OPCAO      ATIVIDADE  TRANSACAO             target
1             Coordenador Geral  SIAPE          FOLHA       ATUASERV                         FPATPEQMSE            1         768
7             Coordenador Geral  SIAPE          SIAPE       GERENCIAL                        GRPROCESSOGRDESBLQRH  4         414
1             Coordenador Geral                                                                                    0         406
3             Chefe                                                                                                0         385
25            Diretor            SIAPE          CADSIAPE    CADASTRO   ATUCADAST             CDALCONPGDN92116W4    11        302
3             Chefe              SIAPE          SIAPENET    ORGAO                            LOGIN                 3         288
8             Coordenador        SIAPE          FOLHA       ATUASERV                         FPCLPAGTO  

In [37]:
df_grupo = df_test[["co_orgao_log","classificacao"]].groupby(["co_orgao_log","classificacao"]).count()

df_grupo.head(20)

KeyError: 'Column not found: target'

In [38]:
df_test["target"].value_count()

AttributeError: 'Series' object has no attribute 'value_count'